In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_songs = pd.read_csv('../data/songs.csv')

In [3]:
df_songs.head()

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,BLACKPINK,TEDDY| FUTURE BOUNCE| Bekuh BOOM,TEDDY,31.0
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,NaN,NaN,31.0
3,dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=,273554,465,S.H.E,湯小康,徐世珍,3.0
4,W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=,140329,726,貴族精選,Traditional,Traditional,52.0


In [4]:
df_songs['song_length_minute'] = df_songs['song_length'] / (60 * 1000)
df_songs['song_length_minute'] = df_songs['song_length_minute'].astype(int)

# 将大于20分钟的时间，分成两个段[20,100]: 20, [100, +oo]: 21; 使用类别特征，进行onehot
df_songs.loc[((df_songs['song_length_minute']>=20) &  (df_songs['song_length_minute']<100)),'song_length_minute'] = 20
df_songs.loc[df_songs['song_length_minute']>=100,'song_length_minute'] = 21

# df_songs['song_length_minute'].value_counts()

In [5]:
# 填充空值
df_songs.loc[df_songs['genre_ids'].isnull(), 'genre_ids'] = df_songs['genre_ids'].value_counts().index[0]
# genre_ids的个数
df_songs['genre_ids_counts'] = df_songs['genre_ids'].apply(lambda x: x.count("|")+1)


In [6]:
# 得到最热门的[artist_name, composer, lyricist], 根据不同的个数，进行排序
df_songs_artist_name= pd.DataFrame(data={"artist_name":df_songs["artist_name"].value_counts().index, "artist_name_counts":df_songs["artist_name"].value_counts().values})
df_songs_artist_name.fillna(0, inplace=True)
df_songs_composer = pd.DataFrame(data={"composer":df_songs["composer"].value_counts().index, "composer_counts":df_songs["composer"].value_counts().values})
df_songs_composer.fillna(0, inplace=True)
df_songs_lyricist = pd.DataFrame(data={"lyricist":df_songs["lyricist"].value_counts().index, "lyricist_counts":df_songs["lyricist"].value_counts().values})
df_songs_lyricist.fillna(0, inplace=True)



In [7]:
df_songs = pd.merge(df_songs, df_songs_artist_name, on='artist_name', how='left')
df_songs = pd.merge(df_songs, df_songs_composer, on='composer', how='left')
df_songs = pd.merge(df_songs, df_songs_lyricist, on='lyricist', how='left')


In [8]:
df_songs.head()

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language,song_length_minute,genre_ids_counts,artist_name_counts,composer_counts,lyricist_counts
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0,4,1,352,3.0,165.0
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,BLACKPINK,TEDDY| FUTURE BOUNCE| Bekuh BOOM,TEDDY,31.0,3,1,10,3.0,8.0
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,NaN,NaN,31.0,3,1,375,NaN,NaN
3,dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=,273554,465,S.H.E,湯小康,徐世珍,3.0,4,1,349,25.0,99.0
4,W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=,140329,726,貴族精選,Traditional,Traditional,52.0,2,1,2419,10105.0,1751.0


In [9]:
def contain2(x, y):
    if not x or not y:
        return 0
    x = str(x)
    y = str(y)
    if x in y or y in x:
        return 1
    return 0


def contain3(x, y, z):
    if not x or not y or not y:
        return 0
    x = str(x)
    y = str(y)
    z = str(z)
    minlen = min(len(x), len(y), len(z))
    if len(x)  == minlen:
        t = x
    elif len(y)  == minlen:
        t = y
    else:
        t = z        
    if t in x and t in y and t in z:
        return 1
    return 0




In [10]:
df_songs['artist_composer_same'] = df_songs[['artist_name','composer']].apply(lambda row: contain2(row['artist_name'], row['composer']), axis=1)
df_songs['artist_lyricist_same'] = df_songs[['artist_name','lyricist']].apply(lambda row: contain2(row['artist_name'], row['lyricist']), axis=1)
df_songs['composer_lyricist_same'] = df_songs[['composer', 'lyricist']].apply(lambda row: contain2(row['composer'], row['lyricist']), axis=1)



In [11]:
df_songs['artist_composer_lyricist_same'] = df_songs[['artist_name', 'composer', 'lyricist']].apply(lambda row: contain3(row['artist_name'], row['composer'], row['lyricist']), axis=1)


In [12]:
print(df_songs['artist_composer_same'].value_counts())
print(df_songs['artist_lyricist_same'].value_counts())
print(df_songs['composer_lyricist_same'].value_counts())
print(df_songs['artist_composer_lyricist_same'].value_counts())


0    2054095
1     242225
Name: artist_composer_same, dtype: int64
0    2224760
1      71560
Name: artist_lyricist_same, dtype: int64
1    1302368
0     993952
Name: composer_lyricist_same, dtype: int64
0    2233061
1      63259
Name: artist_composer_lyricist_same, dtype: int64


In [13]:
df_song_extra = pd.read_csv('../data/song_extra_info.csv')
# 歌名的长度，国家
df_song_extra['song_name_length'] = df_song_extra['name'].apply(lambda x: len(str(x)))
df_song_extra['song_country'] = df_song_extra['isrc'].apply(lambda x: str(x)[:2])


In [14]:
def isrc_to_year(isrc):
    if type(isrc) == str:
        if int(isrc[5:7]) > 17:
            return 1900 + int(isrc[5:7])
        else:
            return 2000 + int(isrc[5:7])
    else:
        return np.nan
        
df_song_extra['song_year'] = df_song_extra['isrc'].apply(isrc_to_year)

In [15]:
df_songs = pd.merge(df_songs, df_song_extra, on='song_id', how='left')
df_songs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2296320 entries, 0 to 2296319
Data columns (total 21 columns):
song_id                          object
song_length                      int64
genre_ids                        object
artist_name                      object
composer                         object
lyricist                         object
language                         float64
song_length_minute               int64
genre_ids_counts                 int64
artist_name_counts               int64
composer_counts                  float64
lyricist_counts                  float64
artist_composer_same             int64
artist_lyricist_same             int64
composer_lyricist_same           int64
artist_composer_lyricist_same    int64
name                             object
isrc                             object
song_name_length                 float64
song_country                     object
song_year                        float64
dtypes: float64(5), int64(8), object(8)
memory usa

In [16]:
df_songs.to_csv('../data/processed/df_songs.csv',index=False)